In [ ]:
library(CIBERSORT)
library(tidyverse)
library(tidyr)
library(psych)
library(ggpubr)
library(ggExtra)
library(cowplot)
library(pheatmap)
tacg_exp = 2^TCGA_risk[,11:17671]-1
entire_exp =  entire_risk[,1:17661]
entire_exp = rownames_to_column(entire_exp)
write.table(entire_exp,file = "entire_exp.txt",row.names = F,quote = F,sep = "\t")
tacg_exp = rownames_to_column(tacg_exp)
write.table(tacg_exp,file = "tacg_exp.txt",row.names = F,quote = F,sep = "\t")
tacg_exp = t(tacg_exp)
entire_exp = t(entire_exp)
results <- cibersort(sig_matrix = LM22, mixture_file = mixed_expr)
tcga_results <- cibersort(sig_matrix = LM22, mixture_file = tacg_exp)
entire_results <- cibersort(sig_matrix = LM22, mixture_file = entire_exp)
load("cibersort_data.rdata")
enntire_group = data.frame(Sample = rownames(entire_results),Group=entire_risk$risk)
tcga_group = data.frame(Sample = rownames(tcga_results),Group=TCGA_risk$risk)
group = enntire_group#tcga_group;enntire_group
results = entire_results#tcga_results;entire_results
library(pheatmap)
re <- results[,-(23:25)]
k <- apply(re,2,function(x) {sum(x == 0) < nrow(results)/2})
re2 <- as.data.frame(t(re[,k]))
##########构造注释文件
an = data.frame(Group = group$Group,
                row.names = group$Sample)
dat <- re %>% as.data.frame() %>%
  rownames_to_column("Sample") %>% 
  gather(key = Cell_type,value = Proportion,-Sample)
dat = data.frame(dat,group)
dat = data.frame(dat,entire_risk[,c("GZMB","TIMP1","SLC39A8",'riskScore')])
g1 = ggplot(dat, aes(x = Sample, y = Proportion, fill = Cell_type)) +
  geom_bar(stat = "identity", position = "stack") +
  facet_grid(~Group, scales = "free_x", space = "free_x", switch = "x") +
  labs(fill = "Cell Type", x = "", y = "Estimated Proportion") +
  theme_bw() +
  theme(axis.text.x = element_blank(),
        axis.ticks.x = element_blank(),
       legend.position = "bottom") +
  scale_y_continuous(expand = c(0.01, 0)) +
  scale_fill_manual(values = mypalette(22))
ggsave("cibersortheadbar.pdf",g1,width = 10,height = 5)
ggplot(dat,aes(Cell_type,Proportion,fill = Cell_type)) + 
geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "Cell Type", y = "Estimated Proportion") +
  theme(axis.text.x = element_blank(),
        axis.ticks.x = element_blank(),
        legend.position = "bottom") + 
  scale_fill_manual(values = mypalette(22))
g2 = ggplot(dat,aes(Cell_type,Proportion,fill = Group)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "Cell Type", y = "Estimated Proportion") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=90,vjust = 0.7, hjust = 1))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
  stat_compare_means(aes(group = Group,label = ..p.signif..),method = "wilcox.test")
ggsave("cibersortboxr.pdf",g2,width = 7,height = 7)
dat_ = spread(dat,Cell_type,Proportion)
dat__ = dat_[,-c(1,2,3)]
rownames(dat_) = dat_[,1]
### 相关性图
d1 = dat_[,8:29];d2 = dat_[,4:7]
phy = d1;met = d2
#计算相关性
cor <-corr.test(phy, met, method = "pearson",adjust="none")
#提取相关性、p值
cmt <-cor$r
pmt <- cor$p
#判断显著性
if (!is.null(pmt)){
ssmt <- pmt< 0.01
pmt[ssmt] <-'**'
smt <- pmt >0.01& pmt <0.05
pmt[smt] <- '*'
pmt[!ssmt&!smt]<- ''
} else {
  pmt <- F
 }
pheatmap(cmt,scale = "none",cluster_row = F, cluster_col = T, border=NA,
      display_numbers = pmt,fontsize_number = 12, number_color = "black",
      cellwidth = 20,treeheight_col= 0, treeheight_row= 0,cellheight =20,color=colorRampPalette(c("#2fa1dd", "white", "#f87669"))(100),filename = "cor_heat.pdf")
pmt[pmt[,4] != "",]%>%rownames()%>%gsub(" ","_",.)->cell
cell[length(cell)] = "T_cells_regulatory"
colnames(dat_) = gsub(" ","_",colnames(dat_))
colnames(dat_)[ncol(dat_)] = "T_cells_regulatory"
theme_global <- theme_bw()+theme(panel.grid.major = element_blank(),panel.grid.minor=element_blank()) + theme(aspect.ratio = 1)+  theme(
    text = element_text(size = 15),  # 调整所有文本的字体大小
    axis.text = element_text(size = 15),  # 调整坐标轴标签的字体大小
    axis.title = element_text(size = 15),  # 调整坐标轴标题的字体大小
    legend.title = element_text(size = 14),  # 调整图例标题的字体大小
    legend.text = element_text(size = 12)  # 调整图例文本的字体大小
  )
dat__ = data.frame(dat_[,1:7],dat_[,cell])
pp = list()
for(i in 8:ncol(dat__)){
g1 = ggscatter(dat__, x =colnames(dat_)[7] ,y = colnames(dat__)[i],
          add = "reg.line", conf.int = T,
          add.params = list(fill = "lightgray"),
          ggtheme = theme_minimal(), margin.plot =  "density"
)+ stat_cor(method = "pearson",
           label.x = median(dat_[,7]), label.y =max(dat_[,i]),color='red',size =7)+theme_global
g2 = ggMarginal(g1, type="density",xparams = list( fill="#3c69ac", size = 0.5,alpha=1), yparams = list(fill="#bb4962", size = 0.5,alpha=1))
pp[[i]] = g2
}
plot_ = pp[8:length(pp)]
pdf("cor_list.pdf",width = 30,height = 12)
plot_grid(plotlist = plot_, align = "h", nrow = 2)
dev.off()
